In [65]:
# import polars as pl
import pandas as pd
import ray
import modin.pandas as md
from pandarallel import pandarallel
import time
import json

# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn import metrics
# from sklearn.ensemble import ExtraTreesClassifier

In [17]:
ray.shutdown()

In [3]:
ray.init()

2022-08-07 22:40:57,666	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:8265


RayContext(dashboard_url='127.0.0.1:8265', python_version='3.9.12', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-08-07_22-40-55_073973_28695/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-08-07_22-40-55_073973_28695/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-08-07_22-40-55_073973_28695', 'metrics_export_port': 63868, 'gcs_address': '127.0.0.1:60247', 'address': '127.0.0.1:60247', 'node_id': '3abf169750c095619c754db4803821e109d3b8c449b5fee3073a12cf'})

In [55]:
%%time
# data = pl.read_parquet("../data/interim/tranco_16_05_22_10k_run_06/part_0.parquet.gzip")

data2 = pd.read_parquet("../data/interim/tranco_16_05_22_10k_run_06/part_0.parquet.gzip")

# data2 = md.read_parquet("../data/interim/tranco_16_05_22_10k_run_06/part_0.parquet.gzip")

CPU times: user 7.01 s, sys: 1.79 s, total: 8.81 s
Wall time: 9.82 s


In [20]:
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [56]:
%%time
data2

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.25 µs


,easylist,easyprivacy,hostname,pathname,filetype,filename,protocol,query,content-type,vary,...,x-odoo-frontend,x-hello-human,x-libsyn-host,x-client-geo-latlong,x-client-geo-country,x-client-geo-city,x-client-geo-region,x-client-geo-metrocode,x-client-geo-zip,mid
0,False,False,csp.withgoogle.com,/csp/uxe-owners-acl/marketing_platform,/csp/uxe-owners-acl/marketing_platform,marketing_platform,https:,[],text/html; charset=utf-8,"sec-fetch-dest, sec-fetch-mode, sec-fetch-site",...,None,None,None,None,None,None,None,None,None,None
1,False,False,ei.phncdn.com,/www-static/js/lib/networkbar-5.0.0.js,js,networkbar-5.0.0.js,https:,"[[cache, 2022051705]]",application/javascript,accept-encoding,...,None,None,None,None,None,None,None,None,None,None
2,False,False,ei.phncdn.com,/www-static/images/pornhub_logo_straight.svg,svg,pornhub_logo_straight.svg,https:,"[[cache, 2022051705]]",image/svg+xml,accept-encoding,...,None,None,None,None,None,None,None,None,None,None
3,False,False,ei.phncdn.com,/videos/202201/09/401031701/original/(m=eafTGg...,jpg,(m=eafTGgaaaa)(mh=wv_2B3Uo7aDPa66D)9.jpg,https:,[],image/jpeg,None,...,None,None,None,None,None,None,None,None,None,None
4,False,False,di.phncdn.com,/videos/202203/14/404605051/original/(m=q3WX_I...,jpg,(m=q3WX_IXbeafTGgaaaa)(mh=yFGzYLUJ7NCGW6_e)0.jpg,https:,[],image/jpeg,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167553,False,False,cdnjs.cloudflare.com,/ajax/libs/angular-ui-router/0.2.10/angular-ui...,js,angular-ui-router.min.js,https:,"[[v, 14.158.0]]",application/javascript; charset=utf-8,accept-encoding,...,None,None,None,None,None,None,None,None,None,None
167554,False,False,www.browsealoud.com,/modules/3.6.1/136.fc673e37f3d700e1ec0d.js,js,136.fc673e37f3d700e1ec0d.js,https:,[],application/javascript,"origin,access-control-request-headers,access-c...",...,None,None,None,None,None,None,None,None,None,None
167555,False,True,www.googletagmanager.com,/gtag/js,/gtag/js,js,https:,"[[id, AW-627293384]]",application/javascript; charset=utf-8,accept-encoding,...,None,None,None,None,None,None,None,None,None,None
167556,False,False,buy.tinypass.com,/ng/common/i18n/platform-translation-map_en_US.js,js,platform-translation-map_en_US.js,https:,"[[version, 14.158.0]]",application/javascript;charset=utf-8,accept-encoding,...,None,None,None,None,None,None,None,None,None,None


In [64]:
data2.info(memory_usage="deep")

# 14GB -> 763MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167558 entries, 0 to 167557
Columns: 3921 entries, easylist to mid
dtypes: bool(2), category(3918), object(1)
memory usage: 763.0 MB


### algorithm:

1. get list of column names
2. list of memory usage per column
3. iterate over all columns
4. change dtype
5. compare memory usage (new dtype) with the old memory usage (old dtype)
6. if new < old then keep change else try another dtype

In [9]:
@ray.remote
def test(data):
    columns = data.columns.values
    old_memory = data.memory_usage()

    for (index, column) in enumerate(columns):
        print(index, column)
        if column == 'query':
            continue
        new_memory = data[column].astype("category").memory_usage(deep=True)
        if new_memory < old_memory[index+1]:
            data[column] = data[column].astype("category")

In [10]:
@ray.remote
def test2(col_name, old_memory):
    index = data2.columns.get_loc(col_name)
    if col_name in ["query"]:
        return
    print(index, col_name)
    new_memory = data2[col_name].astype("category").memory_usage(deep=True)
    if new_memory < old_memory[index+1]:
        # data2[col_name] = data2[col_name].astype("category")
        return col_name

In [8]:
%%time
current_memory = data2.memory_usage(deep=True)
# [print(idx, column) for idx, column in enumerate(data2)]

CPU times: user 221 ms, sys: 93.6 ms, total: 315 ms
Wall time: 10.7 s


In [9]:
# data2.iloc[:,8:14].memory_usage(deep=True)

# data2.iloc[:,8:14].dtypes

current_memory

Index                          128
easylist                    167558
easyprivacy                 167558
hostname                  12566332
pathname                  17070411
                            ...   
x-client-geo-city          4021442
x-client-geo-region        4021433
x-client-geo-metrocode     4021425
x-client-geo-zip           4021430
mid                        4021427
Length: 3922, dtype: int64

In [108]:
def test3(col_name, old_memory):
    index = test.columns.get_loc(col_name)
    if col_name in ["query"]:
        return
    print(index, col_name)
    new_memory = test[col_name].astype("category").memory_usage(deep=True)
    if new_memory < old_memory[index+1]:
        test[col_name] = test[col_name].astype("category")

In [58]:
test = data2.iloc[:,1:14]
test_columns = test.columns.values
test

,easyprivacy,hostname,pathname,filetype,filename,protocol,query,content-type,vary,cache-control,pragma,expires,date
0,False,csp.withgoogle.com,/csp/uxe-owners-acl/marketing_platform,/csp/uxe-owners-acl/marketing_platform,marketing_platform,https:,[],text/html; charset=utf-8,"sec-fetch-dest, sec-fetch-mode, sec-fetch-site","no-cache, no-store, max-age=0, must-revalidate",no-cache,"mon, 01 jan 1990 00:00:00 gmt","wed, 18 may 2022 13:15:41 gmt"
1,False,ei.phncdn.com,/www-static/js/lib/networkbar-5.0.0.js,js,networkbar-5.0.0.js,https:,"[[cache, 2022051705]]",application/javascript,accept-encoding,max-age=10368000,None,"wed, 14 sep 2022 22:07:10 gmt","wed, 18 may 2022 13:15:41 gmt"
2,False,ei.phncdn.com,/www-static/images/pornhub_logo_straight.svg,svg,pornhub_logo_straight.svg,https:,"[[cache, 2022051705]]",image/svg+xml,accept-encoding,max-age=10368000,None,"wed, 14 sep 2022 22:07:09 gmt","wed, 18 may 2022 13:15:41 gmt"
3,False,ei.phncdn.com,/videos/202201/09/401031701/original/(m=eafTGg...,jpg,(m=eafTGgaaaa)(mh=wv_2B3Uo7aDPa66D)9.jpg,https:,[],image/jpeg,None,max-age=9945154,None,"thu, 08 sep 2022 23:39:00 gmt","wed, 18 may 2022 13:15:41 gmt"
4,False,di.phncdn.com,/videos/202203/14/404605051/original/(m=q3WX_I...,jpg,(m=q3WX_IXbeafTGgaaaa)(mh=yFGzYLUJ7NCGW6_e)0.jpg,https:,[],image/jpeg,None,max-age=10466555,None,None,"wed, 18 may 2022 13:15:41 gmt"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
167553,False,cdnjs.cloudflare.com,/ajax/libs/angular-ui-router/0.2.10/angular-ui...,js,angular-ui-router.min.js,https:,"[[v, 14.158.0]]",application/javascript; charset=utf-8,accept-encoding,"public, max-age=30672000",None,"mon, 08 may 2023 15:06:24 gmt","wed, 18 may 2022 15:06:24 gmt"
167554,False,www.browsealoud.com,/modules/3.6.1/136.fc673e37f3d700e1ec0d.js,js,136.fc673e37f3d700e1ec0d.js,https:,[],application/javascript,"origin,access-control-request-headers,access-c...",None,None,None,"wed, 18 may 2022 09:55:09 gmt"
167555,True,www.googletagmanager.com,/gtag/js,/gtag/js,js,https:,"[[id, AW-627293384]]",application/javascript; charset=utf-8,accept-encoding,"private, max-age=900",None,"wed, 18 may 2022 15:06:24 gmt","wed, 18 may 2022 15:06:24 gmt"
167556,False,buy.tinypass.com,/ng/common/i18n/platform-translation-map_en_US.js,js,platform-translation-map_en_US.js,https:,"[[version, 14.158.0]]",application/javascript;charset=utf-8,accept-encoding,"public, max-age=86400",None,"thu, 19 may 2022 15:06:24 gmt","wed, 18 may 2022 15:06:24 gmt"


In [116]:
%%time
test = test.parallel_apply(lambda x: test3(x.name, current_memory), axis=0)

0 content-type
1 vary
2 cache-control3
 pragma
4 expires
5 date
CPU times: user 113 ms, sys: 92.9 ms, total: 206 ms
Wall time: 313 ms


In [134]:
test.memory_usage(deep=True)

704

In [11]:
%%time
category_list = ray.get([test2.remote(column, current_memory) for column in data2])

(test2 pid=28721) 1 easyprivacy
(test2 pid=28713) 3 pathname
(test2 pid=28715) 5 filename
(test2 pid=28714) 2 hostname
(test2 pid=28716) 4 filetype
(test2 pid=28719) 6 protocol
(test2 pid=28722) 8 content-type
(test2 pid=28718) 0 easylist
(test2 pid=28720) 10 cache-control
(test2 pid=28717) 9 vary
(test2 pid=28763) 11 pragma
(test2 pid=28764) 12 expires
(scheduler +5m49s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +5m49s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(test2 pid=28722) 13 date
(test2 pid=28770) 14 accept-ch
(test2 pid=28721) 15 permissions-policy
(test2 pid=28765) 16 content-security-policy
(test2 pid=28771) 17 cross-origin-resource-policy
(test2 pid=28767) 18 report-to
(test2 pid=28763) 19 cross-origin-opener-polic

In [ ]:
%%time
category_list = data2.parallel_apply(lambda x: test2(x.name, current_memory), axis=0)

In [12]:
category_list

[None,
 None,
 'hostname',
 'pathname',
 'filetype',
 'filename',
 'protocol',
 None,
 'content-type',
 'vary',
 'cache-control',
 'pragma',
 'expires',
 'date',
 'accept-ch',
 'permissions-policy',
 'content-security-policy',
 'cross-origin-resource-policy',
 'report-to',
 'cross-origin-opener-policy',
 'server',
 'content-length',
 'x-xss-protection',
 'x-frame-options',
 'x-content-type-options',
 'alt-svc',
 'last-modified',
 'etag',
 'content-encoding',
 'access-control-allow-origin',
 'access-control-allow-methods',
 'timing-allow-origin',
 'x-cdn-diag',
 'accept-ranges',
 'x-hw',
 'cross-origin-opener-policy-report-only',
 'x-guploader-uploadid',
 'x-goog-hash',
 'x-goog-storage-class',
 'via',
 'age',
 'x-served-by',
 'x-cache',
 'x-cache-hits',
 'x-timer',
 'content-disposition',
 'strict-transport-security',
 'p3p',
 'access-control-allow-credentials',
 'google-lineitem-id',
 'google-creative-id',
 'google-mediationgroup-id',
 'google-mediationtag-id',
 'x-via-cdn',
 'x-via-e

In [48]:
category_list_final = [i for i in category_list if i is not None]
category_list_final

['hostname',
 'pathname',
 'filetype',
 'filename',
 'protocol',
 'content-type',
 'vary',
 'cache-control',
 'pragma',
 'expires',
 'date',
 'accept-ch',
 'permissions-policy',
 'content-security-policy',
 'cross-origin-resource-policy',
 'report-to',
 'cross-origin-opener-policy',
 'server',
 'content-length',
 'x-xss-protection',
 'x-frame-options',
 'x-content-type-options',
 'alt-svc',
 'last-modified',
 'etag',
 'content-encoding',
 'access-control-allow-origin',
 'access-control-allow-methods',
 'timing-allow-origin',
 'x-cdn-diag',
 'accept-ranges',
 'x-hw',
 'cross-origin-opener-policy-report-only',
 'x-guploader-uploadid',
 'x-goog-hash',
 'x-goog-storage-class',
 'via',
 'age',
 'x-served-by',
 'x-cache',
 'x-cache-hits',
 'x-timer',
 'content-disposition',
 'strict-transport-security',
 'p3p',
 'access-control-allow-credentials',
 'google-lineitem-id',
 'google-creative-id',
 'google-mediationgroup-id',
 'google-mediationtag-id',
 'x-via-cdn',
 'x-via-edge',
 'x-requestid',

In [50]:
dict_categories = {i : 'category' for i in category_list_final}

dict_categories

{'hostname': 'category',
 'pathname': 'category',
 'filetype': 'category',
 'filename': 'category',
 'protocol': 'category',
 'content-type': 'category',
 'vary': 'category',
 'cache-control': 'category',
 'pragma': 'category',
 'expires': 'category',
 'date': 'category',
 'accept-ch': 'category',
 'permissions-policy': 'category',
 'content-security-policy': 'category',
 'cross-origin-resource-policy': 'category',
 'report-to': 'category',
 'cross-origin-opener-policy': 'category',
 'server': 'category',
 'content-length': 'category',
 'x-xss-protection': 'category',
 'x-frame-options': 'category',
 'x-content-type-options': 'category',
 'alt-svc': 'category',
 'last-modified': 'category',
 'etag': 'category',
 'content-encoding': 'category',
 'access-control-allow-origin': 'category',
 'access-control-allow-methods': 'category',
 'timing-allow-origin': 'category',
 'x-cdn-diag': 'category',
 'accept-ranges': 'category',
 'x-hw': 'category',
 'cross-origin-opener-policy-report-only': 

In [ ]:
with open("dict_categories.json", "w") as f:
    json.dump(dict_categories, f)

In [62]:
%%time
data2 = data2.astype(dict_categories)

CPU times: user 21.4 s, sys: 2 s, total: 23.4 s
Wall time: 24.8 s


In [52]:
test.apply(lambda x: x.astype("category") if x.name in category_list_final else False, axis=0)

easyprivacy                                                  False
hostname         0               csp.withgoogle.com
1          ...
pathname         0                    /csp/uxe-owners-acl/marke...
filetype         0         /csp/uxe-owners-acl/marketing_platfo...
filename         0                                       market...
protocol         0         https:
1         https:
2         ht...
query                                                        False
content-type     0                      text/html; charset=utf-...
vary             0            sec-fetch-dest, sec-fetch-mode, s...
cache-control    0         no-cache, no-store, max-age=0, must-...
pragma           0         no-cache
1              NaN
2       ...
expires          0         mon, 01 jan 1990 00:00:00 gmt
1     ...
date             0         wed, 18 may 2022 13:15:41 gmt
1     ...
dtype: object

In [63]:
data2.dtypes

easylist                      bool
easyprivacy                   bool
hostname                  category
pathname                  category
filetype                  category
                            ...   
x-client-geo-city         category
x-client-geo-region       category
x-client-geo-metrocode    category
x-client-geo-zip          category
mid                       category
Length: 3921, dtype: object

## Final approach:
#### 1. current_memory = data2.memory_usage(deep=True)
#### -- moding df --
#### 2. category_list = ray.get([test2.remote(column, current_memory) for column in data2])
#### 3. category_list_final = [i for i in category_list if i is not None]
#### 4. dict_categories = {i : 'category' for i in category_list_final}
#### 5. with open("dict_categories.json", "w") as f:
####      json.dump(dict_categories, f)
#### -- pandas df --
#### 6. data2 = data2.astype(dict_categories)

In [69]:
data2.groupby(["easylist", "easyprivacy"]).count()

hostname  pathname  filetype  filename  protocol  \
easylist easyprivacy                                                     
False    False          122092    122092    122092    122092    122092   
         True            28379     28379     28379     28379     28379   
True     False           14073     14073     14073     14073     14073   
         True             3014      3014      3014      3014      3014   

                       query  content-type   vary  cache-control  pragma  ...  \
easylist easyprivacy                                                      ...   
False    False        122092        120511  68674         106403    5498  ...   
         True          28379         25314  13221          20862    8658  ...   
True     False         14073         13547   5741          11845    4608  ...   
         True           3014          2719    345           2349    1786  ...   

                      x-odoo-frontend  x-hello-human  x-libsyn-host  \
easylist easyprivacy                                                  
False    False                     50              2              2   
         True                       0              0              0   
True     False                      0              0              0   
         True                       0              0              0   

                      x-client-geo-latlong  x-client-geo-country  \
easylist easyprivacy                                               
False    False                           1                     1   
         True                            0                     0   
True     False                           0                     0   
         True                            0                     0   

                      x-client-geo-city  x-client-geo-region  \
easylist easyprivacy                                           
False    False                        1                    1   
         True                         0                    0   
True     False                        0                    0   
         True                         0                    0   

                      x-client-geo-metrocode  x-client-geo-zip  mid  
easylist easyprivacy                                                 
False    False                             1                 1    0  
         True                              0                 0    1  
True     False                             0                 0    0  
         True                              0                 0    0  

[4 rows x 3919 columns]

In [70]:
(data2.isna().sum() / len(data2)) >= 0.95

easylist                  False
easyprivacy               False
hostname                  False
pathname                  False
filetype                  False
                          ...  
x-client-geo-city          True
x-client-geo-region        True
x-client-geo-metrocode     True
x-client-geo-zip           True
mid                        True
Length: 3921, dtype: bool

In [71]:
data2.groupby("filetype", maintain_order=True).count()

TypeError: groupby() got an unexpected keyword argument 'maintain_order'

In [37]:
data2.groupby("filetype", maintain_order=True).count()

filetype,count
str,u32
"""/csp/uxe-owner...",14
"""js""",41130
"""svg""",9124
"""jpg""",22324
"""html""",2073
"""css""",8623
"""/_/Subscribewi...",2
"""/gampad/ads""",260
"""/JjewRHousCsko...",1
